In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# /deploy/edge_inference.py
"""
Edge Inference Script for Operation FractureScope
Detects micro-defects (crack, corrosion, scratch) from multimodal images
Optimized for resource-constrained edge devices
"""

import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import time


class FractureNet(nn.Module):
    def __init__(self, num_classes=3):
        super(FractureNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),  # Lightweight layers
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.classifier = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# === Preprocessing (resize + normalize) ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# === Load model weights ===
model = FractureNet(num_classes=3)
model.load_state_dict(torch.load('fracturenet.pth', map_location='cpu'))
model.eval()

# === Class labels ===
classes = ['crack', 'corrosion', 'scratch']

def edge_predict(image_path):
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)  # Shape: (1, 3, 224, 224)
    
    start_time = time.time()
    with torch.no_grad():
        output = model(img_tensor)
        probs = torch.nn.functional.softmax(output, dim=1)
        pred_idx = torch.argmax(probs, dim=1).item()
        pred_class = classes[pred_idx]
        confidence = probs[0, pred_idx].item()
    inference_time = (time.time() - start_time) * 1000  # ms
    
    return {
        'prediction': pred_class,
        'confidence': round(confidence, 3),
        'inference_time_ms': round(inference_time, 2)
    }

# === Example Usage ===
if __name__ == '__main__':
    img_path = 'sample_edge_image.jpg'  # Replace with actual edge-captured image
    result = edge_predict(img_path)
    print(f"[Edge Inference] Class: {result['prediction']} | Confidence: {result['confidence']} | Time: {result['inference_time_ms']} ms")
